## **PurpleAir Monitors** ##

In [2]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot
import pandas as pd

In [3]:
# Load cleaned data
purple_df = pd.read_csv('../data/clean_purpleair.csv')
purple_df.head()

,time,location_name,location_id,latitude,longitude,pm2_5_1h_mean,pm2_5_1h_mean_aqi,pm2_5_24h_mean,pm2_5_24h_mean_aqi,temp,rh,elevation,pressure
0,2018-12-27 04:00:00,Grundy Park,21427,37.622585,-122.42097,0.736345,4.0,2.999879,16.0,53.318182,59.818182,86.0,NaN
1,2018-12-27 05:00:00,Grundy Park,21427,37.622585,-122.42097,0.739827,4.0,2.999879,16.0,51.777778,59.955556,86.0,NaN
2,2018-12-27 06:00:00,Grundy Park,21427,37.622585,-122.42097,1.038868,6.0,2.999879,16.0,52.068182,56.681818,86.0,NaN
3,2018-12-27 07:00:00,Grundy Park,21427,37.622585,-122.42097,1.214613,7.0,2.999879,16.0,52.755556,56.933333,86.0,NaN
4,2018-12-27 08:00:00,Grundy Park,21427,37.622585,-122.42097,1.127572,6.0,2.999879,16.0,65.883721,54.372093,86.0,NaN


### **Preprocessing Check**

In [4]:
# Convert longitude and latitude to correct types
purple_df['longitude'] = purple_df['longitude'].apply(lambda x: -abs(x))
purple_df['latitude'] = purple_df['latitude'].apply(lambda x: abs(x))

# Convert time to datetime (if not already done)
purple_df['time'] = pd.to_datetime(purple_df['time'], format='%Y-%m-%d %H:%M:%S')

purple_df.head()

,time,location_name,location_id,latitude,longitude,pm2_5_1h_mean,pm2_5_1h_mean_aqi,pm2_5_24h_mean,pm2_5_24h_mean_aqi,temp,rh,elevation,pressure
0,2018-12-27 04:00:00,Grundy Park,21427,37.622585,-122.42097,0.736345,4.0,2.999879,16.0,53.318182,59.818182,86.0,NaN
1,2018-12-27 05:00:00,Grundy Park,21427,37.622585,-122.42097,0.739827,4.0,2.999879,16.0,51.777778,59.955556,86.0,NaN
2,2018-12-27 06:00:00,Grundy Park,21427,37.622585,-122.42097,1.038868,6.0,2.999879,16.0,52.068182,56.681818,86.0,NaN
3,2018-12-27 07:00:00,Grundy Park,21427,37.622585,-122.42097,1.214613,7.0,2.999879,16.0,52.755556,56.933333,86.0,NaN
4,2018-12-27 08:00:00,Grundy Park,21427,37.622585,-122.42097,1.127572,6.0,2.999879,16.0,65.883721,54.372093,86.0,NaN


In [5]:
# Create column for date only
purple_df['date'] = purple_df['time'].dt.date
purple_df['date'] = pd.to_datetime(purple_df['date'], format='%Y-%m-%d')

purple_df['date'].head()

0   2018-12-27
1   2018-12-27
2   2018-12-27
3   2018-12-27
4   2018-12-27
Name: date, dtype: datetime64[ns]

### **Visualizing and Analyzing**

In [6]:
# Only take unique dates and their average pm2.5 concentrations
unique_dates = purple_df.drop_duplicates(subset=['location_name', 'date'], keep='first')

In [7]:
# Find outliers using IQR and separate them from the dataframe
def find_outliers_iqr(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    non_outliers = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return non_outliers, outliers

# Remove outliers and keep them in a separate dataframe
unique_dates, outliers_df = find_outliers_iqr(unique_dates, 'pm2_5_24h_mean')

print("Number of outliers removed:", len(outliers_df))
print("Number of unique dates remaining:", len(unique_dates))

Number of outliers removed: 2104
Number of unique dates remaining: 24034


In [ ]:
# Create figure with subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Without Outliers", "Outliers"))

# Add histogram for PM2.5 values without outliers
fig.add_trace(
    go.Histogram(
        x=unique_dates['pm2_5_24h_mean'],
        nbinsx=35,
        marker=dict(color='blue'),
        hoverinfo='x+y'
    ),
    row=1, col=1
)

# Add histogram for PM2.5 values outliers
fig.add_trace(
    go.Box(
        x=outliers_df['pm2_5_24h_mean'],
        marker=dict(color='green'),
        hoverinfo='x',
        name=''
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(title_text="Distribution of PM2.5 Values for PurpleAir Monitors",
                  showlegend=False)
fig.update_xaxes(title_text="Daily Average PM2.5 Concentration (µg/m³)", row=1, col=1)
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_xaxes(title_text="Daily Average PM2.5 Concentration (µg/m³)", row=1, col=2)

# Save the figure offline
plot(fig, filename='../figures/purpleair_pm25_distribution.html', auto_open=True)

'../figures/purpleair_pm25_distribution.html'

In [8]:
# Calculate the rolling 24h mean for PM2.5 for each hour
purple_df['pm2_5_24h_rolling_mean'] = purple_df.groupby('location_name')['pm2_5_1h_mean'].transform(
    lambda x: x.rolling(window=24, min_periods=1).mean()
)

# Impute missing values in the rolling mean
purple_df['pm2_5_24h_rolling_mean'] = purple_df['pm2_5_24h_rolling_mean'].ffill()
purple_df['pm2_5_24h_rolling_mean'] = purple_df['pm2_5_24h_rolling_mean'].bfill()

purple_df.head()

,time,location_name,location_id,latitude,longitude,pm2_5_1h_mean,pm2_5_1h_mean_aqi,pm2_5_24h_mean,pm2_5_24h_mean_aqi,temp,rh,elevation,pressure,date,pm2_5_24h_rolling_mean
0,2018-12-27 04:00:00,Grundy Park,21427,37.622585,-122.42097,0.736345,4.0,2.999879,16.0,53.318182,59.818182,86.0,NaN,2018-12-27,0.736345
1,2018-12-27 05:00:00,Grundy Park,21427,37.622585,-122.42097,0.739827,4.0,2.999879,16.0,51.777778,59.955556,86.0,NaN,2018-12-27,0.738086
2,2018-12-27 06:00:00,Grundy Park,21427,37.622585,-122.42097,1.038868,6.0,2.999879,16.0,52.068182,56.681818,86.0,NaN,2018-12-27,0.838347
3,2018-12-27 07:00:00,Grundy Park,21427,37.622585,-122.42097,1.214613,7.0,2.999879,16.0,52.755556,56.933333,86.0,NaN,2018-12-27,0.932413
4,2018-12-27 08:00:00,Grundy Park,21427,37.622585,-122.42097,1.127572,6.0,2.999879,16.0,65.883721,54.372093,86.0,NaN,2018-12-27,0.971445


In [9]:
# AQI calculation function
def calculate_pm2_5_aqi(C_p):
    if pd.isna(C_p):
        return None

    C_p = float(str(C_p)[:str(C_p).find('.')+2]) if '.' in str(C_p) else float(C_p)

    breakpoints = [
        (0.0,   9.0,   0,   50),
        (9.1,   35.4,  51,  100),
        (35.5,  55.4,  101, 150),
        (55.5,  125.4, 151, 200),
        (125.5, 225.4, 201, 300),
        (225.5, 500.4, 301, 500)
    ]

    for BP_Lo, BP_Hi, I_Lo, I_Hi in breakpoints:
        if BP_Lo <= C_p <= BP_Hi:
            I_p = ((I_Hi - I_Lo) / (BP_Hi - BP_Lo)) * (C_p - BP_Lo) + I_Lo
            return round(I_p)

    return None

# Calculate AQI for PM2.5 24hr mean
purple_df['pm2_5_24h_rolling_mean_aqi'] = purple_df['pm2_5_24h_rolling_mean'].apply(calculate_pm2_5_aqi)

In [10]:
# Find outliers from dataframe
no_outliers_df, outliers_df = find_outliers_iqr(purple_df, 'pm2_5_24h_rolling_mean_aqi')

# Create figure with subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Without Outliers", "Outliers"))

# Scatterplot of PM2.5 values for PurpleAir monitors without outliers
fig.add_trace(
        go.Scatter(
                x=no_outliers_df['pm2_5_1h_mean'],
                y=no_outliers_df['pm2_5_24h_rolling_mean_aqi'],
                mode='markers',
                marker=dict(color='blue', size=5),
                hovertext=no_outliers_df['location_name'],
                name=''
        ),
        row=1, col=1
)

# Scatterplot of PM2.5 values for PurpleAir monitors with outliers
fig.add_trace(
        go.Scatter(
                x=outliers_df['pm2_5_1h_mean'],
                y=outliers_df['pm2_5_24h_rolling_mean_aqi'],
                mode='markers',
                marker=dict(color='red', size=5),
                hovertext=outliers_df['location_name'],
                name=''
        ),
        row=1, col=2
)

# Update axis titles
fig.update_xaxes(title_text="Hourly Average PM2.5 Concentration (µg/m³)", row=1, col=1)
fig.update_yaxes(title_text="Daily Average PM2.5 AQI", row=1, col=1)
fig.update_xaxes(title_text="Hourly Average PM2.5 Concentration (µg/m³)", row=1, col=2)
fig.update_yaxes(title_text="Daily Average PM2.5 AQI", row=1, col=2)

# Update layout
fig.update_layout(title_text="Scatter Plots of PM2.5 Values for PurpleAir Monitors",
                                  showlegend=False)

# Save the figure offline
plot(fig, filename='../figures/purpleair_pm25_aqi_scatter.html', auto_open=True)

'../figures/purpleair_pm25_aqi_scatter.html'

## **Clarity Monitors**

In [9]:
# Load cleaned data for Clarity monitors
clarity_df = pd.read_csv('../data/clean_clarity.csv')

# Sort by monitor and time
clarity_df = clarity_df.sort_values(['location_name', 'time'])

# Convert longitude and latitude to correct types
clarity_df['longitude'] = clarity_df['longitude'].apply(lambda x: -abs(x))
clarity_df['latitude'] = clarity_df['latitude'].apply(lambda x: abs(x))

# Convert time to datetime (if not already done)
clarity_df['time'] = pd.to_datetime(clarity_df['time'], format='%Y-%m-%d %H:%M:%S')

# Create column for date only
clarity_df['date'] = clarity_df['time'].dt.date
clarity_df['date'] = pd.to_datetime(clarity_df['date'], format='%Y-%m-%d')

clarity_df.head()

,time,location_name,location_id,latitude,longitude,pm2_5_1h_mean,pm2_5_1h_mean_aqi,pm2_5_24h_mean,pm2_5_24h_mean_aqi,temp,rh,date
33,2024-10-31 11:00:00,Belle Air School,DRCAC7970,37.62441,-122.40469,4.07,22.0,3.942308,22,20.00,54.91,2024-10-31
34,2024-10-31 12:00:00,Belle Air School,DRCAC7970,37.62441,-122.40469,4.11,23.0,3.942308,22,20.50,51.66,2024-10-31
49,2024-10-31 13:00:00,Belle Air School,DRCAC7970,37.62441,-122.40469,4.38,24.0,3.942308,22,20.47,50.51,2024-10-31
52,2024-10-31 14:00:00,Belle Air School,DRCAC7970,37.62441,-122.40469,4.15,23.0,3.942308,22,19.94,53.11,2024-10-31
63,2024-10-31 15:00:00,Belle Air School,DRCAC7970,37.62441,-122.40469,3.67,20.0,3.942308,22,18.86,56.11,2024-10-31


In [ ]:
# Only take unique dates and their average pm2.5 concentrations
unique_dates_clarity = clarity_df.drop_duplicates(subset=['location_name', 'date'], keep='first')

# Find outliers using IQR and separate them from the dataframe
unique_dates_clarity, outliers_df_clarity = find_outliers_iqr(unique_dates_clarity, 'pm2_5_24h_mean')

# Create figure with subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Without Outliers", "Outliers"))

# Add histogram for PM2.5 values without outliers
fig.add_trace(
    go.Histogram(
        x=unique_dates_clarity['pm2_5_24h_mean'],
        nbinsx=35,
        marker=dict(color='blue'),
        hoverinfo='x+y'
    ),
    row=1, col=1
)

# Add histogram for PM2.5 values outliers
fig.add_trace(
    go.Box(
        x=outliers_df_clarity['pm2_5_24h_mean'],
        marker=dict(color='green'),
        hoverinfo='x',
        name=''
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(title_text="Distribution of PM2.5 Values for Clarity Monitors",
                  showlegend=False)
fig.update_xaxes(title_text="Daily Average PM2.5 Concentration (µg/m³)", row=1, col=1)
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_xaxes(title_text="Daily Average PM2.5 Concentration (µg/m³)", row=1, col=2)

# Save the figure offline
plot(fig, filename='../figures/clarity_pm25_distribution.html', auto_open=True)

In [12]:
# Calculate the rolling 24h mean for PM2.5 for each hour
clarity_df['pm2_5_24h_rolling_mean'] = clarity_df.groupby('location_name')['pm2_5_1h_mean'].transform(
    lambda x: x.rolling(window=24, min_periods=1).mean()
)

# Impute missing values in the rolling mean
clarity_df['pm2_5_24h_rolling_mean'] = clarity_df['pm2_5_24h_rolling_mean'].ffill()
clarity_df['pm2_5_24h_rolling_mean'] = clarity_df['pm2_5_24h_rolling_mean'].bfill()

# Calculate AQI for PM2.5 24hr mean
clarity_df['pm2_5_24h_rolling_mean_aqi'] = clarity_df['pm2_5_24h_rolling_mean'].apply(calculate_pm2_5_aqi)

# Find outliers from dataframe
no_outliers_df_clarity, outliers_df_clarity = find_outliers_iqr(clarity_df, 'pm2_5_24h_rolling_mean_aqi')

# Create figure with subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Without Outliers", "Outliers"))

# Scatterplot of PM2.5 values for Clarity monitors without outliers
fig.add_trace(
        go.Scatter(
                x=no_outliers_df_clarity['pm2_5_1h_mean'],
                y=no_outliers_df_clarity['pm2_5_24h_rolling_mean_aqi'],
                mode='markers',
                marker=dict(color='blue', size=5),
                hovertext=no_outliers_df_clarity['location_name'],
                name=''
        ),
        row=1, col=1
)

# Scatterplot of PM2.5 values for Clarity monitors with outliers
fig.add_trace(
        go.Scatter(
                x=outliers_df_clarity['pm2_5_1h_mean'],
                y=outliers_df_clarity['pm2_5_24h_rolling_mean_aqi'],
                mode='markers',
                marker=dict(color='red', size=5),
                hovertext=outliers_df_clarity['location_name'],
                name=''
        ),
        row=1, col=2
)

# Update axis titles
fig.update_xaxes(title_text="Hourly Average PM2.5 Concentration (µg/m³)", row=1, col=1)
fig.update_yaxes(title_text="Daily Average PM2.5 AQI", row=1, col=1)
fig.update_xaxes(title_text="Hourly Average PM2.5 Concentration (µg/m³)", row=1, col=2)
fig.update_yaxes(title_text="Daily Average PM2.5 AQI", row=1, col=2)

# Update layout
fig.update_layout(title_text="Scatter Plots of PM2.5 Values for Clarity Monitors",
                                  showlegend=False)

# Save the figure offline
plot(fig, filename='../figures/clarity_pm25_aqi_scatter.html', auto_open=True)

'../figures/clarity_pm25_aqi_scatter.html'

In [13]:
# Keep the clarity_df_grouped definition
clarity_df_grouped = clarity_df.groupby(['date']).agg({'pm2_5_1h_mean': 'mean'}).reset_index()
clarity_df_grouped['month_year_str'] = clarity_df_grouped['date'].dt.strftime('%B')
clarity_df_grouped['day'] = clarity_df_grouped['date'].dt.day

# Create a facet grid with month-year as the facet column
fig = px.line(
    clarity_df_grouped,
    x="day",
    y="pm2_5_1h_mean",
    facet_col="month_year_str",
    facet_col_wrap=6,
    title="PM2.5 Concentrations by Month (Clarity Monitors)",
    labels={"pm2_5_1h_mean": "PM2.5 (1-hour mean)", "day": "Day of Month", "month_year_str": "Month"},
    height=400
)

# Update layout for better visualization
fig.update_layout(
    margin=dict(t=50, l=50, r=50, b=50),
    title_x=0.5
)

# Show the plot
fig.show()


## **Self-Prediction**

### **Experiment: Clarity Monitors**

In [24]:
# Filter the data
filtered_df = clarity_df[['location_id', 'location_name', 'date', 'time', 'pm2_5_24h_mean']]
filtered_df.head()

,location_id,location_name,date,time,pm2_5_24h_mean
33,DRCAC7970,Belle Air School,2024-10-31,2024-10-31 11:00:00,3.942308
34,DRCAC7970,Belle Air School,2024-10-31,2024-10-31 12:00:00,3.942308
49,DRCAC7970,Belle Air School,2024-10-31,2024-10-31 13:00:00,3.942308
52,DRCAC7970,Belle Air School,2024-10-31,2024-10-31 14:00:00,3.942308
63,DRCAC7970,Belle Air School,2024-10-31,2024-10-31 15:00:00,3.942308


In [25]:
# Round the pm2_5_24h_mean values to 2 decimal places
filtered_df['pm2_5_24h_mean'] = filtered_df['pm2_5_24h_mean'].round(2)
filtered_df.head()

/var/folders/tc/ygtvb05s0279mq2rg4cgkqp00000gn/T/ipykernel_16364/3412733926.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,location_id,location_name,date,time,pm2_5_24h_mean
33,DRCAC7970,Belle Air School,2024-10-31,2024-10-31 11:00:00,3.94
34,DRCAC7970,Belle Air School,2024-10-31,2024-10-31 12:00:00,3.94
49,DRCAC7970,Belle Air School,2024-10-31,2024-10-31 13:00:00,3.94
52,DRCAC7970,Belle Air School,2024-10-31,2024-10-31 14:00:00,3.94
63,DRCAC7970,Belle Air School,2024-10-31,2024-10-31 15:00:00,3.94


In [26]:
# Remove outliers from data (either removing, or just imputing)


In [27]:
# Get the unique values by date
unique_dates_filtered = filtered_df.drop_duplicates(subset=['location_name', 'date'], keep='first')
unique_dates_filtered.head()

,location_id,location_name,date,time,pm2_5_24h_mean
33,DRCAC7970,Belle Air School,2024-10-31,2024-10-31 11:00:00,3.94
150,DRCAC7970,Belle Air School,2024-11-01,2024-11-01 00:00:00,4.89
361,DRCAC7970,Belle Air School,2024-11-02,2024-11-02 00:00:00,4.02
574,DRCAC7970,Belle Air School,2024-11-03,2024-11-03 00:00:00,4.34
803,DRCAC7970,Belle Air School,2024-11-04,2024-11-04 00:00:00,5.06


In [28]:
unique_dates_filtered.shape

(2069, 5)

In [29]:
# Distribution of PM2.5 values for PurpleAir monitors
px.histogram(unique_dates_filtered[unique_dates_filtered['pm2_5_24h_mean'] <= 200], 
             x='pm2_5_24h_mean', title='Distribution of PM2.5 24hr Mean Values', nbins=50).show()

In [30]:
import numpy as np

# Distribution of the pm2_5_24h_mean values
# Apply log transformation to the pm2_5_24h_mean values
unique_dates_filtered['pm2_5_24h_mean_log'] = unique_dates_filtered['pm2_5_24h_mean'].apply(lambda x: np.log(x + 1))
px.histogram(unique_dates_filtered, x='pm2_5_24h_mean_log', title='Distribution of Log PM2.5 24hr Mean Values', nbins=25).show()

/var/folders/tc/ygtvb05s0279mq2rg4cgkqp00000gn/T/ipykernel_16364/2184446324.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [31]:
# Create bins for the log pm2_5_24h_mean values and get edges of the bins
bins = pd.qcut(unique_dates_filtered['pm2_5_24h_mean_log'], q=10)

# Get the edges of the bins
edges = bins.cat.categories
edges = [(edges[i].left, edges[i].right) for i in range(len(edges))]
edges = [(edges[i][0].round(2), edges[i][1].round(2)) for i in range(len(edges))]
edges = [(np.exp(edges[i][0]) - 1, np.exp(edges[i][1]) - 1) for i in range(len(edges))]

edges

[(1.691234472349262, 3.3059595283452063),
 (3.3059595283452063, 3.854955811237434),
 (3.854955811237434, 4.259310844446898),
 (4.259310844446898, 4.754602676005731),
 (4.754602676005731, 5.359819522601832),
 (5.359819522601832, 6.170676488346613),
 (6.170676488346613, 7.331137487687693),
 (7.331137487687693, 9.59095145243378),
 (9.59095145243378, 12.873769902129904),
 (12.873769902129904, 34.163197145106615)]

In [32]:
# Create final edges for the bins
final_edges = [(0, 3.50), (3.51, 4.50), (4.51, 5.50), (5.51, 7.01), (7.01, 10.50), (10.51, 15.00), (15.01, 35.00)]

# Create a new column for the bins
unique_dates_filtered['pm2_5_24h_mean_bins'] = pd.cut(unique_dates_filtered['pm2_5_24h_mean'], 
                                                      bins=[x[0] for x in final_edges] + [final_edges[-1][1]], labels=[f"{i}" for i in range(len(final_edges))], 
                                                      include_lowest=True)

unique_dates_filtered.head()

/var/folders/tc/ygtvb05s0279mq2rg4cgkqp00000gn/T/ipykernel_16364/3381559572.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,location_id,location_name,date,time,pm2_5_24h_mean,pm2_5_24h_mean_log,pm2_5_24h_mean_bins
33,DRCAC7970,Belle Air School,2024-10-31,2024-10-31 11:00:00,3.94,1.597365,1
150,DRCAC7970,Belle Air School,2024-11-01,2024-11-01 00:00:00,4.89,1.773256,2
361,DRCAC7970,Belle Air School,2024-11-02,2024-11-02 00:00:00,4.02,1.613430,1
574,DRCAC7970,Belle Air School,2024-11-03,2024-11-03 00:00:00,4.34,1.675226,1
803,DRCAC7970,Belle Air School,2024-11-04,2024-11-04 00:00:00,5.06,1.801710,2


In [33]:
# Get distribution of the bins using proportions
bins_distribution = unique_dates_filtered['pm2_5_24h_mean_bins'].value_counts(normalize=True).sort_index()
bins_distribution = bins_distribution.reset_index()
bins_distribution.columns = ['pm2_5_24h_mean_bins', 'proportion']

# Create bar plot for the bins distribution
fig = px.bar(bins_distribution, x='pm2_5_24h_mean_bins', y='proportion', 
             title='Distribution of PM2.5 24hr Mean Values by Bins', 
             labels={'pm2_5_24h_mean_bins': 'PM2.5 24hr Mean Bins', 'proportion': 'Proportion'},
             color='proportion', color_continuous_scale=px.colors.sequential.Plasma)

fig.show()

In [34]:
# Create a new dataframe with rolling window of 7 days and the next day's bin as label
window_size = 7

# Group by location_id to ensure the rolling window is applied per location
data = []
for location_id, group in unique_dates_filtered.groupby('location_id'):
    group = group.sort_values('date').reset_index(drop=True)
    
    for i in range(len(group) - window_size):
        # Extract the 7-day window of pm2_5_24h_mean
        features = group.loc[i:i+window_size-1, 'pm2_5_24h_mean'].values.tolist()
        # Extract the next day's bin as the label
        label = group.loc[i+window_size, 'pm2_5_24h_mean_bins']
        # Include location_id and location_name
        location_id_value = group.loc[i, 'location_id']
        location_name_value = group.loc[i, 'location_name']
        data.append([location_id_value, location_name_value] + features + [label])

# Create the final dataframe
columns = ['location_id', 'location_name'] + [f'feature_day_{i+1}' for i in range(window_size)] + ['label']
rolling_window_df = pd.DataFrame(data, columns=columns)

rolling_window_df.head()

,location_id,location_name,feature_day_1,feature_day_2,feature_day_3,feature_day_4,feature_day_5,feature_day_6,feature_day_7,label
0,DCVIM2201,Brentwood Park,4.31,3.87,4.00,5.58,5.92,4.67,4.17,2
1,DCVIM2201,Brentwood Park,3.87,4.00,5.58,5.92,4.67,4.17,4.68,0
2,DCVIM2201,Brentwood Park,4.00,5.58,5.92,4.67,4.17,4.68,2.91,0
3,DCVIM2201,Brentwood Park,5.58,5.92,4.67,4.17,4.68,2.91,1.86,0
4,DCVIM2201,Brentwood Park,5.92,4.67,4.17,4.68,2.91,1.86,1.83,0


In [35]:
# Tranform all feature_day columns with a log transformation
# rolling_window_df.iloc[:, 2:-1] = rolling_window_df.iloc[:, 2:-1].apply(lambda x: np.log(x + 1))
# rolling_window_df.head()

In [36]:
rolling_window_df.shape

(1971, 10)

In [37]:
# Build LSTM model and dataset

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)  # Add a channel dimension
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class AirForecastClassifier(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, num_classes=7):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # lstm_out: (batch, seq_len, hidden_dim)
        final_hidden = lstm_out[:, -1, :]  # take last timestep
        logits = self.fc(final_hidden)
        return logits

In [38]:
def train_model(model, dataloader, epochs=10, lr=1e-3):
    """
    AirForecast LSTM Classifier model training function.
    Args:
        model (nn.Module): The LSTM model to train.
        dataloader (DataLoader): DataLoader for the training data.
        epochs (int): Number of training epochs.
        lr (float): Learning rate for the optimizer.
    Returns:
        model (nn.Module): The trained LSTM model.
    """
    # Use available device
    device = torch.device("mps" if torch.backends.mps.is_available()
                          else "cuda" if torch.cuda.is_available()
                          else "cpu")
    model.to(device)

    # Loss and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Forward pass
            logits = model(X_batch)               # (batch_size, num_classes)
            loss = criterion(logits, y_batch)     # y_batch: (batch_size,)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Metrics
            epoch_loss += loss.item()
            preds = logits.argmax(dim=1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

        avg_loss = epoch_loss / len(dataloader)
        accuracy = correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2%}")

    return model

def evaluate(model, dataloader):
    """
    AirForecast LSTM model evaluation function.
    Args:
        model (nn.Module): The LSTM model to evaluate.
        dataloader (DataLoader): DataLoader for the validation data.
    Returns:
        preds (np.ndarray): Predicted class labels.
        truths (np.ndarray): True labels.
    """
    model.eval()
    preds, truths = [], []

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            logits = model(X_batch)
            batch_preds = logits.argmax(dim=1)

            preds.append(batch_preds.cpu().numpy())
            truths.append(y_batch.cpu().numpy())

    preds = np.concatenate(preds)
    truths = np.concatenate(truths)
    accuracy = np.mean(preds == truths)

    print(f"Validation Accuracy: {accuracy:.2%}")
    return preds, truths

In [39]:
# Ensure all monitors are included in the validation set using location_index_map
val_indices = []

for location, group in rolling_window_df.groupby('location_id'):
    split_index = int(len(group) * 0.8)
    if split_index < len(group):  # Ensure split_index is within bounds
        val_indices.extend(group.index[split_index:])

val_indices = np.array(val_indices)
train_indices = np.setdiff1d(rolling_window_df.index, val_indices)

train_df = rolling_window_df.loc[train_indices]
val_df = rolling_window_df.loc[val_indices]

# Create TimeSeriesDataset instances for training and validation sets
train_dataset = TimeSeriesDataset(train_df.iloc[:, 2:-1].values, train_df['label'].astype(int).values)
val_dataset = TimeSeriesDataset(val_df.iloc[:, 2:-1].values, val_df['label'].astype(int).values)

# Create DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

# Train the model
model = AirForecastClassifier(input_dim=1, num_classes=7)
trained_model = train_model(model, train_loader, epochs=20)

Epoch 1/20, Loss: 1.9052, Accuracy: 21.09%
Epoch 2/20, Loss: 1.7998, Accuracy: 27.00%
Epoch 3/20, Loss: 1.7190, Accuracy: 28.59%
Epoch 4/20, Loss: 1.6310, Accuracy: 31.96%
Epoch 5/20, Loss: 1.5816, Accuracy: 35.26%
Epoch 6/20, Loss: 1.5501, Accuracy: 35.71%
Epoch 7/20, Loss: 1.5327, Accuracy: 36.85%
Epoch 8/20, Loss: 1.5142, Accuracy: 37.04%
Epoch 9/20, Loss: 1.4982, Accuracy: 38.95%
Epoch 10/20, Loss: 1.4953, Accuracy: 37.80%
Epoch 11/20, Loss: 1.4933, Accuracy: 37.93%
Epoch 12/20, Loss: 1.4757, Accuracy: 39.90%
Epoch 13/20, Loss: 1.4548, Accuracy: 40.28%
Epoch 14/20, Loss: 1.4433, Accuracy: 41.23%
Epoch 15/20, Loss: 1.4410, Accuracy: 40.15%
Epoch 16/20, Loss: 1.4165, Accuracy: 41.04%
Epoch 17/20, Loss: 1.4049, Accuracy: 43.52%
Epoch 18/20, Loss: 1.3947, Accuracy: 43.01%
Epoch 19/20, Loss: 1.3736, Accuracy: 44.85%
Epoch 20/20, Loss: 1.3622, Accuracy: 46.51%


In [40]:
# Evaluate the model by monitor location
# Move the trained model back to CPU
trained_model = trained_model.to('cpu')

# Evaluate the model by monitor location
predictions, truths = evaluate(trained_model, val_loader)

Validation Accuracy: 26.45%


In [41]:
# Find the accuracy by monitor location
location_accuracy = {}
for location_id in rolling_window_df['location_id'].unique():
    location_indices = rolling_window_df[rolling_window_df['location_id'] == location_id].index
    location_preds = predictions[np.isin(val_indices, location_indices)]
    location_truths = truths[np.isin(val_indices, location_indices)]
    accuracy = np.mean(location_preds == location_truths)
    location_accuracy[location_id] = accuracy
    print(f"Location {location_id}: Accuracy = {accuracy:.2%}")


Location DCVIM2201: Accuracy = 18.52%
Location DETMG3939: Accuracy = 17.24%
Location DEVPF7186: Accuracy = 20.69%
Location DHPSP8686: Accuracy = 33.33%
Location DHSHV3008: Accuracy = 40.00%
Location DJGNN5114: Accuracy = 31.03%
Location DJTYV8538: Accuracy = 31.03%
Location DMEYT2138: Accuracy = 22.22%
Location DNSEJ7404: Accuracy = 22.22%
Location DRCAC7970: Accuracy = 27.59%
Location DRYLF3821: Accuracy = 17.24%
Location DTMSK2119: Accuracy = 37.93%
Location DUBTA4581: Accuracy = 31.03%
Location DVRGV9737: Accuracy = 18.52%


In [ ]:
from sklearn.model_selection import KFold

# Create k-fold cross-validation sets
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Split the rolling_window_df into k folds
folds = []
for train_index, val_index in kf.split(rolling_window_df):
    train_set = rolling_window_df.iloc[train_index].reset_index(drop=True)
    val_set = rolling_window_df.iloc[val_index].reset_index(drop=True)
    
    # Ensure each training and validation set includes all monitors
    if (set(train_set['location_id'].unique()) == set(rolling_window_df['location_id'].unique()) and
        set(val_set['location_id'].unique()) == set(rolling_window_df['location_id'].unique())):
        folds.append((train_set, val_set))

len(folds)

5

In [61]:
# Create RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create a list to store fold accuracy for each monitor
monitor_fold_accuracies = []

# Train and evaluate the Decision Tree Classifier for each fold
for i, (train_set, val_set) in enumerate(folds):
    # Extract features and labels
    X_train = train_set.iloc[:, 2:-1].values
    y_train = train_set['label'].astype(int).values
    X_val = val_set.iloc[:, 2:-1].values
    y_val = val_set['label'].astype(int).values

    # Create and train the Decision Tree Classifier
    clf = RandomForestClassifier(random_state=42, criterion='entropy')
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_val)

    # Calculate accuracy for each monitor
    for location_id in val_set['location_id'].unique():
        location_indices = val_set[val_set['location_id'] == location_id].index
        location_y_val = y_val[np.isin(val_set.index, location_indices)]
        location_y_pred = y_pred[np.isin(val_set.index, location_indices)]
        location_accuracy = accuracy_score(location_y_val, location_y_pred)

        # Store the results
        monitor_fold_accuracies.append({
            'fold': i + 1,
            'location_id': location_id,
            'accuracy': location_accuracy
        })

# Convert the results to a DataFrame
monitor_fold_accuracies_df = pd.DataFrame(monitor_fold_accuracies)

# Display the DataFrame
monitor_fold_accuracies_df.head()


,fold,location_id,accuracy
0,1,DCVIM2201,0.714286
1,1,DETMG3939,0.793103
2,1,DEVPF7186,0.812500
3,1,DHPSP8686,0.518519
4,1,DHSHV3008,0.500000


In [66]:
# Find average accuracy by fold by monitor location for random forest classifier
average_accuracy_by_fold = monitor_fold_accuracies_df.groupby(['location_id']).agg({'accuracy': 'mean'}).reset_index()
average_accuracy_by_fold = average_accuracy_by_fold.sort_values(by='accuracy', ascending=False)

average_accuracy_by_fold.head(10)

,location_id,accuracy
1,DETMG3939,0.789522
0,DCVIM2201,0.769736
7,DMEYT2138,0.755608
2,DEVPF7186,0.755303
10,DRYLF3821,0.750265
12,DUBTA4581,0.724753
11,DTMSK2119,0.713212
9,DRCAC7970,0.712444
13,DVRGV9737,0.687551
8,DNSEJ7404,0.677146


In [67]:
average_accuracy_by_fold.shape

(14, 2)